In [1]:
# import package
import os
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
import math
import sys
import random
import csv

In [3]:
# read GroupTopic.txt
gt_lines = []
with open('GroupTopic.txt') as gt_f:
    gt_lines = gt_f.readlines()
gt_f.close()

group_id = []
group_organizer = []
group_attribute = []
gt_count = 1
for gt_line in gt_lines:
    if gt_count % 2 == 1:
      gt_id, gt_organizer = gt_line.split()
      group_id.append(int(gt_id[1:]))
      if gt_organizer == 'null':
        group_organizer.append(int(-1))
      else:
        group_organizer.append(int(gt_organizer[1:]))
    else:
      read_one_group_attribute = gt_line.split()
      num_group_attribute_list = []
      for i in range(len(read_one_group_attribute)):
        num_group_attribute_list.append(int(read_one_group_attribute[i][1:]))
      group_attribute.append(num_group_attribute_list)
    gt_count += 1

gt_data = list(zip(group_id, group_organizer, group_attribute))
gt_df = pd.DataFrame(gt_data, columns = ['group_id', 'group_organizer', 'group_attribute'])
gt_df = gt_df[gt_df['group_organizer'] != -1]
gt_df = gt_df.drop('group_organizer', 1)
max_group_id = gt_df['group_id'].max()
group_amount = len(gt_df.index)
gt_df

,group_id,group_attribute
0,0,"[85, 89, 124, 234, 738, 2527, 3007, 3135, 5915..."
1,1,"[212, 512, 3554]"
2,2,[1566]
3,3,"[244, 309, 1322, 1502, 3336, 4422, 8649, 9816,..."
4,4,"[638, 9753, 9772, 15067, 15984, 18874, 19061]"
5,5,[825]
6,6,"[211, 1573]"
7,7,"[491, 498, 588, 1091, 1104, 10581, 15117, 1666..."
8,8,"[554, 9792]"
9,9,"[498, 705, 1074, 1091, 1104, 3250, 15117, 1583..."


In [5]:
# read MemberTopic.txt
mt_lines = []
with open('MemberTopic.txt') as mt_f:
    mt_lines = mt_f.readlines()
mt_f.close()

member_id = []
member_attribute = []
mt_count = 1
for mt_line in mt_lines:
    if mt_count % 2 == 1:
      member_id.append(int(mt_line[1:-1]))
    else:
      read_one_member_attribute = mt_line.split()
      num_member_attribute_list = []
      for i in range(len(read_one_member_attribute)):
        num_member_attribute_list.append(int(read_one_member_attribute[i][1:]))
      member_attribute.append(num_member_attribute_list)
    mt_count += 1

mt_data = list(zip(member_id, member_attribute))
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df = mt_df[mt_df.astype(str)['member_attribute'] != '[]']
max_member_id = mt_df['member_id'].max()
member_amount = len(mt_df.index)
mt_df

,member_id,member_attribute
0,0,"[10485, 498]"
1,1,"[1201, 914, 933, 638, 590, 726, 242, 1724, 158..."
2,2,"[10495, 9753, 491, 1201, 638, 2926, 15892, 229..."
3,3,"[11660, 11046, 2151]"
4,4,"[1201, 933, 230, 2899, 1755, 3282, 1923, 228, ..."
6,6,"[1726, 98876, 218321, 218331, 218341]"
9,9,"[17110, 15236, 1044, 4392, 15381, 27700, 1998]"
10,10,[3361]
12,12,"[24460, 21133, 4422, 392, 264, 17951, 10581, 2..."
15,15,"[29791, 29793, 217, 23199, 23960, 381]"


In [7]:
# read Group-Event folder
group_event_id = []
event_id = []
event_limit = []
event_time = []
event_organizer = []
yes_id = []
no_id = []
maybe_id = []
root = "./GroupEvent"
for i in range(0, 482):
  file = "G" + str(i) + ".txt"
  full_path = os.path.join(root, file)
  ge_group_id = file[:-4]
  with open(full_path) as ge_f:
    ge_lines = ge_f.readlines()
  ge_f.close()
  ge_count = 0
  for ge_line in ge_lines:
    ge_count += 1
    if ge_count % 5 == 0:
      maybe_id.append(ge_line.split())
    elif ge_count % 4 == 0:
      read_one_no_id = ge_line.split()
      num_no_id_list = []
      for i in range(len(read_one_no_id)):
        if read_one_no_id[i] != 'null':
          num_no_id_list.append(int(read_one_no_id[i][1:]))
      no_id.append(num_no_id_list)
    elif ge_count % 3 == 0:
      read_one_yes_id = ge_line.split()
      num_yes_id_list = []
      for i in range(len(read_one_yes_id)):
        if read_one_yes_id[i] != 'null':
          num_yes_id_list.append(int(read_one_yes_id[i][1:]))
      yes_id.append(num_yes_id_list)
    elif ge_count % 2 == 0:
      event_organizer.append(ge_line.split())
    else:
      ge_event_id, ge_event_limit, ge_event_time = ge_line.split()
      event_id.append(int(ge_event_id[1:]))
      event_limit.append(ge_event_limit)
      event_time.append(ge_event_time)
      group_event_id.append(int(ge_group_id[1:]))
    if ge_count == 5:
      ge_count = 0

ge_data = list(zip(group_event_id, event_id, event_limit, event_time, event_organizer, yes_id, no_id, maybe_id))
ge_df = pd.DataFrame(ge_data, columns = ['group_id', 'event_id', 'event_limit', 'event_time', 'event_organizer', 'yes_id', 'no_id', 'maybe_id'])
ge_df[['event_limit', 'event_time']] = ge_df[['event_limit', 'event_time']].astype(int)

# remove column maybe_id
ge_df = ge_df.drop('maybe_id', 1)

# remove useless  columns 
ge_df = ge_df.drop('event_organizer', 1)
ge_df = ge_df.drop('event_limit', 1)
ge_df = ge_df.drop('no_id', 1)
ge_df = ge_df.drop('event_time', 1)
ge_df = ge_df[ge_df.astype(str)['yes_id'] != '[]']
max_event_id = ge_df['event_id'].max()
event_amount = len(ge_df.index) 
ge_df

,group_id,event_id,yes_id
0,0,932,[48462]
1,0,947,"[48462, 56494, 62049, 70873, 48571]"
2,0,1742,[48462]
3,0,3372,"[48462, 55059, 56494, 70873]"
4,0,3373,"[48462, 21224, 27288, 63893]"
5,0,6675,"[31584, 48462]"
6,0,7373,"[48462, 62746, 70873]"
7,0,7797,"[31584, 48462, 61831, 2526]"
8,0,7904,"[48462, 70873, 21224]"
9,0,9068,"[48462, 3911]"


In [8]:
# randomly shuffle the rows of ge_df
ge_df = ge_df.sample(frac=1)

# split ge_df into 5 parts
p1, p2, p3, p4, p5 = np.array_split(ge_df, 5)

# train_set_1, test_set_1
train_frames = [p2, p3, p4, p5]
train_set = pd.concat(train_frames)
test_set = p1
train_set

,group_id,event_id,yes_id
4417,53,7371,"[35944, 36222, 47706, 48964, 57436, 57441, 605..."
28236,277,9398,"[34432, 33132, 36083, 39901]"
4555,57,53612,"[56784, 1174, 11436]"
47625,407,48790,"[42094, 43205, 46607, 25086]"
35063,311,34819,[38568]
12290,168,40064,"[36130, 36785, 36958, 37780, 38630, 59116, 619..."
36221,317,43543,"[31712, 35112, 47206, 3325, 4603, 10710, 11082..."
19423,220,19707,"[34166, 41217, 67214, 23342]"
2661,29,7834,"[54324, 61404]"
22920,241,31850,"[21699, 26633, 28105]"


In [20]:
# train_member_set
train_member_set = set()
train_amount = len(train_set.index)
for i in range(train_amount):
  yes_id = train_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  train_member_set.update(yes_id)
len(train_member_set)

52899

In [21]:
# test_member_set
test_member_set = set()
test_amount = len(test_set.index)
for i in range(test_amount):
  yes_id = test_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  test_member_set.update(yes_id)
len(test_member_set)

26825

In [22]:
# member_set
member_set = set()
member_amount = len(mt_df.index)
for i in range(member_amount):
  member_id = mt_df.iloc[i]['member_id']
  member_set.add(member_id)
len(member_set)

47289

In [23]:
# common_member_set, train & test & mt_df
train_test_common_set = list(set(train_member_set).intersection(test_member_set))
common_member_set = list(set(train_test_common_set).intersection(member_set))
common_member_set = [int(i) for i in common_member_set]
len(common_member_set)

14498

In [25]:
# update mt_df, 32s
member_id = common_member_set
member_attribute = [set() for i in range(len(member_id))]
for i in range(len(mt_df.index)):
  id = int(mt_df.iloc[i]['member_id'])
  if id in member_id:
    index =  member_id.index(id)
    attribute = set(mt_df.iloc[i]['member_attribute'])
    member_attribute[index] = attribute

mt_data = list(zip(member_id, member_attribute))
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df

,member_id,member_attribute
0,65541,"{579, 4392, 8652, 498, 2516, 2294, 638}"
1,32776,"{9349, 392, 3336, 264, 16011, 16270, 7319, 151..."
2,9,"{15236, 4392, 1998, 1044, 15381, 17110, 27700}"
3,10,{3361}
4,65552,"{3009, 16516, 78565, 455, 206, 8304, 23473, 45..."
5,20,"{17674, 15117, 914, 20245, 20246, 28824, 17306..."
6,21,"{132161, 148579, 136645, 563, 10579, 182, 7768..."
7,65556,"{2529, 6786, 10306, 1322, 2513, 242, 10002, 10..."
8,65558,"{713, 8652, 1998, 1201, 1459, 638}"
9,32793,{2369}


In [26]:
# group_attribute_set
group_amount = len(gt_df.index)
group_attribute_set = []
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  for j in range(len(one_group_attribute)):
    if one_group_attribute[j] not in group_attribute_set:
      group_attribute_set.append(one_group_attribute[j])
group_attribute_set = set(group_attribute_set)
print(len(group_attribute_set))

1132


In [27]:
# member_attribute_set
member_amount = len(mt_df.index)
member_attribute_set = set()
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  member_attribute_set.update(one_member_attribute)
print(len(member_attribute_set))

10790


In [28]:
# group&member attribute
gm_attribute = group_attribute_set.intersection(member_attribute_set)
len(gm_attribute)

1082

In [30]:
# remove member_attribute (not in gm_attribute), and update mt_df
new_member_id = []
new_member_attribute = []
member_amount = len(mt_df.index)
for i in range(member_amount):
  new_member_id.append(mt_df.iloc[i]['member_id'])
  one_old_member_attribute = mt_df.iloc[i]['member_attribute']
  one_new_member_attribute = gm_attribute.intersection(one_old_member_attribute)
  new_member_attribute.append(one_new_member_attribute)

mt_data = list(zip(new_member_id, new_member_attribute))
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df

,member_id,member_attribute
0,65541,"{579, 4392, 8652, 498, 2516, 2294, 638}"
1,32776,"{9349, 392, 3336, 264, 16011, 16270, 7319, 439..."
2,9,"{15236, 4392, 1998, 1044, 15381}"
3,10,{3361}
4,65552,"{3009, 78565, 455, 206, 23473, 659, 2516, 7029..."
5,20,"{17674, 15117, 914, 20246, 15261, 1566, 542, 3..."
6,21,"{10579, 563, 182, 31254, 17627}"
7,65556,"{10306, 1322, 2513, 242, 10002, 1074, 9753, 638}"
8,65558,"{713, 8652, 1998, 1201, 1459, 638}"
9,32793,{}


In [32]:
# remove group_attribute (not in gm_attribute), and update gt_df
new_group_id = []
new_group_attribute = []
group_amount = len(gt_df.index)
for i in range(group_amount):
  new_group_id.append(gt_df.iloc[i]['group_id'])
  one_old_group_attribute = gt_df.iloc[i]['group_attribute']
  one_new_group_attribute = gm_attribute.intersection(one_old_group_attribute)
  new_group_attribute.append(one_new_group_attribute)

gt_data = list(zip(new_group_id, new_group_attribute))
gt_df = pd.DataFrame(gt_data, columns = ['group_id', 'group_attribute'])
gt_df

,group_id,group_attribute
0,0,"{3007, 738, 23842, 44612, 234, 19119, 16816, 1..."
1,1,"{512, 3554, 212}"
2,2,{1566}
3,3,"{19168, 10306, 4422, 3336, 8649, 1322, 10290, ..."
4,4,"{9772, 15984, 19061, 9753, 18874, 15067, 638}"
5,5,{825}
6,6,"{211, 1573}"
7,7,"{1091, 17030, 491, 588, 15117, 1104, 498, 1666..."
8,8,"{9792, 554}"
9,9,"{705, 1091, 18339, 17030, 16330, 17866, 15117,..."


In [33]:
# re-caculate member_attribute_set
member_amount = len(mt_df.index)
member_attribute_set = set()
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  member_attribute_set.update(one_member_attribute)
print(len(member_attribute_set))

1082


In [34]:
# re-caculate group_attribute_set
group_amount = len(gt_df.index)
group_attribute_set = set()
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  group_attribute_set.update(one_group_attribute)
print(len(group_attribute_set))

1082


In [36]:
# update train_set(rm member not in common_member_set)
common_member_set = set(common_member_set)
new_yes_id = []
train_set_amount = len(train_set.index)
# pirnt(train)
for i in range(train_set_amount):
  yes_id = train_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  yes_id = common_member_set.intersection(yes_id)
  new_yes_id.append(yes_id)
group_id = train_set['group_id']
event_id = train_set['event_id']
train_data = list(zip(group_id, event_id, new_yes_id))
train_set = pd.DataFrame(train_data, columns = ['group_id', 'event_id', 'yes_id'])
train_set

,group_id,event_id,yes_id
0,53,7371,"{57441, 97, 60515, 35944, 3339, 47706, 66486, ..."
1,277,9398,"{34432, 33132, 39901}"
2,57,53612,"{56784, 1174}"
3,407,48790,{42094}
4,311,34819,{}
5,168,40064,"{38630, 59116, 36785, 63667, 37780, 36958}"
6,317,43543,"{31712, 11082, 4603, 13332}"
7,220,19707,"{41217, 34166}"
8,29,7834,{54324}
9,241,31850,{28105}


In [38]:
# matrix UTS， 8min 4s
member_amount = len(mt_df.index)
column_amount = 1
np_mt = mt_df.values
UTS = np.zeros((member_amount, member_amount), dtype=float)
for i in range(member_amount):
  i_attribute = np_mt[i][1]
  for j in range(column_amount, member_amount):
    j_attribute = np_mt[j][1]
    com_attribute = i_attribute.intersection(j_attribute)
    numer = float(len(com_attribute))
    denom = float(len(i_attribute) + len(j_attribute) - len(com_attribute))
    if denom != 0:
      UTS[i][j] = numer / denom
      UTS[j][i] = UTS[i][j]
  column_amount += 1

In [41]:
# member, even_set, 3min 7s
train_amount = len(train_set.index)
member_amount = len(mt_df.index)
index = [*range(member_amount)]
member_id = mt_df['member_id']
dictionary = dict(zip(index, member_id))
np_train = train_set.values
member_event_set = [set() for i in range(member_amount)]
for i in range(train_amount):
  member_set = np_train[i][2]
  for j in member_set:
    member_index = -1
    for key, value in dictionary.items():
      if j == value:
        member_index = key
        break
    member_event_set[member_index].add(np_train[i][1])

me_data = list(zip(member_id, member_event_set))
me_set = pd.DataFrame(me_data, columns = ['member_id', 'member_event_set'])
me_set

,member_id,member_event_set
0,65541,"{39785, 19787, 14156, 54883}"
1,32776,"{28025, 29044, 33686, 36822, 39833, 34650, 33143}"
2,9,"{40227, 4559, 5259, 14199}"
3,10,"{4650, 4037}"
4,65552,"{45313, 137, 33553, 21523, 3238, 3241, 15151, ..."
5,20,{3537}
6,21,"{32869, 19686, 18150, 43525, 38633, 33738, 435..."
7,65556,"{608, 11655, 41578, 395, 40203, 42645, 41079, ..."
8,65558,"{37443, 42885, 33833, 20268, 42193, 42194}"
9,32793,{40534}


In [43]:
# matrix Sim, 11min 34s
member_amount = len(mt_df.index)
column_amount = 1
np_me = me_set.values
Sim = np.zeros((member_amount, member_amount), dtype=float)
for i in range(member_amount):
  i_event = np_me[i][1]
  for j in range(column_amount, member_amount):
    j_event = np_me[j][1]
    com_event = i_event.intersection(j_event)
    numer = float(len(com_event))
    denom = float(len(i_event) + len(j_event) - len(com_event))
    Sim[i][j] += UTS[i][j]
    if denom != 0:
      Sim[i][j] += (numer / denom)
    Sim[j][i] = Sim[i][j]
  column_amount += 1

In [44]:
# Neighbor(a), 2min 38s, attention!!!! is a_index, not a_id
member_amount = len(mt_df.index)
index = [*range(member_amount)]
K = 10
neighbor = [set() for i in range(member_amount)]
for i in range(member_amount):
  index_Sim = zip(index, Sim[i])
  nonzero_index_Sim = [i for i in index_Sim if i[1] > 0]
  if len(nonzero_index_Sim) >= K:
    sorted_index_Sim = sorted(nonzero_index_Sim, key=lambda x: x[1], reverse=True)
    for j in range(K):
      neighbor[i].add(sorted_index_Sim[j][0])

In [46]:
# update test_set(rm member not in common_member_set)
common_member_set = set(common_member_set)
new_yes_id = []
test_set_amount = len(test_set.index)
for i in range(test_set_amount):
  yes_id = test_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  yes_id = common_member_set.intersection(yes_id)
  new_yes_id.append(yes_id)
group_id = test_set['group_id']
event_id = test_set['event_id']
test_data = list(zip(group_id, event_id, new_yes_id))
test_set = pd.DataFrame(test_data, columns = ['group_id', 'event_id', 'yes_id'])
test_set

,group_id,event_id,yes_id
0,17,55338,{31280}
1,340,32714,"{46336, 12833, 18594, 20322, 10468, 47173, 202..."
2,215,42192,"{31649, 62277, 50220, 54030, 24405, 52469, 592..."
3,231,25554,"{21824, 25634, 34882, 27874, 30691, 22791, 252..."
4,469,19402,"{16553, 60262}"
5,447,49573,"{41384, 38410, 55052, 47286, 42714}"
6,201,25318,"{34865, 25787, 33638}"
7,309,11861,"{49505, 41122, 37, 39237, 69957, 66856, 29834,..."
8,168,11411,"{69154, 7970, 8034, 72932, 47398, 26639, 67987..."
9,187,18608,"{64, 23682, 354, 26950, 9320, 5290, 66282, 221..."


In [48]:
# test_event, test_even_attibute
group_id = []
for i in range(len(gt_df.index)):
  group_id.append(gt_df.iloc[i]['group_id'])
test_amount = len(test_set.index)
group_attribute = gt_df['group_attribute']
event_attribute = [set() for i in range(test_amount)]
even_id = test_set['event_id']
for i in range(test_amount):
  one_group_id = test_set.iloc[i]['group_id']
  if one_group_id in group_id:
    group_index = group_id.index(one_group_id)
    one_event_attribute = group_attribute[group_index]
    event_attribute[i] = one_event_attribute

test_et_data = list(zip(even_id, event_attribute))
test_et_set = pd.DataFrame(test_et_data, columns = ['even_id', 'event_attribute'])
test_et_set

,even_id,event_attribute
0,55338,"{18340, 16729, 18873, 9772, 9753, 15279, 15984..."
1,32714,"{4392, 185, 4446, 1903}"
2,42192,"{10209, 781, 17742, 659, 10102, 1238}"
3,25554,"{2513, 106, 10306, 4422}"
4,19402,{2926}
5,49573,"{9792, 954}"
6,25318,"{5945, 726, 590}"
7,11861,"{2272, 16996, 4422, 18854, 4392, 713, 17866, 1..."
8,11411,"{933, 17030, 15832, 16330, 17866, 60589, 242, ..."
9,18608,{1201}


In [49]:
# matrix member-event, UE_test, 57s
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
index = [*range(member_amount)]
dictionary = dict(zip(index, mt_df['member_id']))
UE_test = np.zeros((member_amount, event_amount), dtype=int)
for i in range(event_amount):
  member_id = test_set.iloc[i]['yes_id']
  for j in member_id:
    for key, value in dictionary.items():
      if j == value:
        UE_test[key][i] = 1
        break

In [ ]:
# matrix pred
threshold = 0.5
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
pred = np.zeros((member_amount, event_amount), dtype=float)
for i in range(member_amount):
  i_attribute = mt_df.iloc[i]['member_attribute']
  i_neighbor = neighbor[i]
  sum_R_a = len(UE_test[i].nonzero())
  R_a = float(sum_R_a) / event_amount
  for j in range(event_amount):
    one_event_attribute = event_attribute[j]
    pred_numer = 0
    pred_denom = 0
    behind_plus = 0
    for n in i_neighbor:
      n_neighbor_attribute = mt_df.iloc[n]['member_attribute']
      n_neighbor_attribute.update(i_attribute)
      com_attribute = n_neighbor_attribute.intersection(one_event_attribute)
      if len(com_attribute) != 0:
        pred_numer += Sim[i][n]
      pred_denom += Sim[i][n]
      if pred_denom != 0:
        behind_plus = pred_numer / pred_denom
  pred[i][j] = R_a + behind_plus  
  if pred[i][j] > threshold:
    pred[i][j] = 1
  else:
    pred[i][j] = 0

In [ ]:
# matrix member-event, UE_test
UE_test = lil_matrix((max_member_id+1, max_event_id+1), dtype=np.int8)
for i in range(len(test_set_1)):
  UE_test_c = test_set_1.iloc[i]['event_id']
  UE_test_r_set = test_set_1.iloc[i]['yes_id']
  for j in range(len(UE_test_r_set)):
    UE_test_r = UE_test_r_set[j]
    UE_test[UE_test_r, UE_test_c] = 1
UE_test.toarray()

In [ ]:
# member_set and test_event_set
member_set = []
for i in range(member_amount):
  member_set.append(mt_df.iloc[i]['member_id'])
test_event_set = []
test_set_amount = len(test_set_1.index)
for i in range(test_set_amount):
  test_event_set.append(test_set_1.iloc[i]['event_id'])

In [ ]:
# evaluate result
UE_predict = pred
TP = 0
FP = 0
FN = 0
TN = 0
for i in range(member_amount):
  i_member_id = mt_df.iloc[i]['member_id']
  for j in range(test_set_amount):
    j_event_id = test_set_1.iloc[j]['event_id']
    if UE_predict[i, j] == 1:
      if UE_test[i, j] == 1:
        TP += 1
      else:
        FP += 1
    else:
      if UE_test[i, j] == 1:
        FN += 1
      else:
        TN += 1
Precision = 0
Recall = 0
Accuracy = 0
F1_score = 0
if TP+FP != 0:
  Precision = TP / (TP+FP)
if TP+FN != 0:
  Recall  =TP / (TP+FN)
if TP+TN+FP+FN != 0:
  Accuracy = (TP+TN) / (TP+TN+FP+FN)
if Precision+Recall != 0:
  F1_score = 2 * Precision * Recall / (Precision+Recall)